In [1]:
import sys
sys.path.insert(1, '../code_v4')

In [39]:
import numpy as np
import itertools
from   pauli            import pauli_action,sigma_matrices
from   binary_functions import Bas2Int,Int2Bas
from   numpy            import linalg as LA
from   scipy            import linalg as SciLA
from   tools            import print_state,fidelity,dump_state,read_state,dump_lanz_vecs
from   hamiltonian      import Hmat,Hmoms,Hpsi,Hii

# ----- term-by-term ITE

def H_alpha_psi(H_,psi_,alpha_):
    (A,h,imp,gmp) = H_[alpha_] 
    phi = np.zeros(psi_.shape,dtype=complex)
    for m in np.where(np.abs(h)>1e-8)[0]:
        phi += h[m]*gmp[m,imp[m,:]]*psi_[imp[m,:]]
    return phi.copy()

def ExpmbH_alpha(H_,psi_,alpha_,db):
    phi = psi_.copy()
    chi = psi_.copy()
    i   = 0
    while(LA.norm(chi)>1e-8):
        chi  = (-db/float(i+1))*H_alpha_psi(H_,chi,alpha_)
        phi += chi.copy()
        i   += 1
    nu = LA.norm(phi)
    return phi.copy()/nu,nu

# ----- unitary evolution

def xP_psi(x_,psi_,imp_,gmp_):
    phi = np.zeros(psi_.shape,dtype=complex)
    for m in np.where(np.abs(x_)>1e-8)[0]:
        phi += x_[m]*gmp_[m,imp_[m,:]]*psi_[imp_[m,:]]
    return phi.copy()

def Exp_ixP(x_,psi_,imp_,gmp_):
    phi = psi_.copy()
    chi = psi_.copy()
    i   = 0
    while(LA.norm(chi)>1e-8):
        chi  = (1j/float(i+1))*xP_psi(x_,chi,imp_,gmp_)
        phi += chi.copy()
        i   += 1
    nu = LA.norm(phi)
    return phi.copy()/nu

def QITE_step(H_,psi_,db,xv,check):

    import time

    nalpha = len(H_)
    dn_    = 1.0

    if(xv is None):
        xv = []
        for alpha in range(nalpha):
            (A,h,imp,gmp) = H_[alpha]
            nact = imp.shape[0] 
            xv.append(np.zeros(nact))

    for alpha in range(nalpha):
        # ----- target state

        t0  = time.time()
        delta_alpha,dnalpha_ = ExpmbH_alpha(H_,psi_,alpha,db)
        delta_alpha         -= psi_.copy()
        dn_                 *= dnalpha_

        # ----- pauli action
        (A,h,imp,gmp) = H_[alpha]
        nact = imp.shape[0]
        Pmu_psi = np.zeros(imp.shape,dtype=complex)
        
        for m in range(nact):
            Pmu_psi[m,:] = gmp[m,imp[m,:]]*psi_[imp[m,:]]
            
        t1  = time.time()

        # ----- set linear system 
        Amat = np.dot(np.conj(Pmu_psi),Pmu_psi.T)
        Amat = 2.0*np.real(Amat)

        t2  = time.time()
        bvec = np.dot(Pmu_psi,np.conj(delta_alpha))
        bvec = -2.0*np.imag(bvec)
        t3  = time.time()

        if(check):
            x = SciLA.lstsq(Amat,bvec)[0]
        else:
            zct  = np.dot(bvec,Amat)
            def cost_fun(vct):
                return LA.norm(np.dot(Amat,vct)-bvec)**2
            def J_cost_fun(vct):
                wct = np.dot(Amat,vct)
                wct = np.dot(Amat.T,wct)
                return 2.0*(wct-zct)
            import scipy
            x    = scipy.optimize.minimize(cost_fun,x0=xv[alpha],method='Newton-CG',jac=J_cost_fun,tol=1e-8).x
        xv[alpha] = x.copy()

        t4  = time.time()
        psi_ = Exp_ixP(x,psi_,imp,gmp)
        t5  = time.time()

        #print alpha,t5-t4,t4-t3,t3-t2,t2-t1,t1-t0
        import sys
        sys.stdout.flush()

    return psi_,dn_,xv

def Lanczos_QITE(hv,sv,db):
    nv = len(range(0,len(hv),2))
    hm = np.zeros((nv,nv),dtype=complex)
    sm = np.zeros((nv,nv),dtype=complex)
    for jr in range(0,len(hv),2):
        for js in range(0,len(hv),2):
            jk = (jr+js)//2
            sm[jr//2,js//2] = np.exp(2*sv[jk]-sv[jr]-sv[js])
            hm[jr//2,js//2] = hv[jk]*sm[jr//2,js//2]

    # rarefied sampling
    idx = []
    for l in range(nv):
        if(absint(np.sqrt(2.0)**l)<nv+1): idx.append(l)
    if(nv-1 not in idx): idx.append(nv-1)
    sm = sm[idx,:]
    sm = sm[:,idx]
    hm = hm[idx,:]
    hm = hm[:,idx]
    nv = sm.shape[0]

    # regularization
    for jk in range(nv):
        sm[jk,jk] *= 1.0+2*db
        hm[jk,jk] *= 1.0+2*db

    eps,U = SciLA.eigh(hm,sm)
    eps   = np.real(eps)
    return np.min(eps)

def QITE(H_,db,bmax,lanczos=False,psi0=None,omega=None,ncheck=1,davidson=True):

    if(davidson):
        N     = H_[0][2].shape[1]
        nbit  = int(np.log2(N))
        hdiag = np.zeros(N,dtype=complex)
        for i in range(N):
            hdiag[i] = Hii(H_,i)
            print(i,hdiag[i])

        precond = lambda x,e, *args: x/(hdiag-e+1e-4)

        def hop(c_):
            return Hpsi(H_,c_)

        if(psi0 is None):
            i0       = np.argmin(hdiag)
            psi0     = np.zeros(N,dtype=complex)
            psi0[i0] = 1.0

        from pyscf.lib import davidson
        epsm0,Um0 = davidson(hop,psi0,precond)
    
    else:
        Hm    = Hmat(H_)
        N     = Hm.shape[0]
        nbit  = int(np.log2(N))
        eps,U = SciLA.eigh(Hm)
        m0    = np.argmin(eps)
        epsm0 = eps[m0]
        Um0   = U[:,m0]
        zeta  = np.exp(-db*(eps-eps[m0]))
        fide  = 1.0

    print("epsm0: ", epsm0)
    fout = open('QITE.out','w')
    fout.write("FCI gs energy %.6f \n" % epsm0)
    fout.write("FCI gs wfn \n")
    print_state(Um0,nbit,fout)

    psi_QITE = psi0[:]

    nbeta = int(bmax/db)+1
    hvect_LANZ = np.zeros(nbeta+1)
    svect_LANZ = np.zeros(nbeta+1)

    xv = None
    fout.write("QITE\n")
    for ib in range(nbeta):
        ea,ev           = Hmoms(H_,psi_QITE)
        hvect_LANZ[ib]  = ea

        if(omega is None): fide = fidelity(psi_QITE,Um0)
        else:              fide = LA.norm(psi_QITE[omega])**2

        if(lanczos):
            ea_            = Lanczos_QITE(hvect_LANZ[:ib+1],svect_LANZ[:ib+1],db) 
            fout.write("%.6f %.6f %.6f %.6f %.6f \n" % (ib*db,ea,ev,fide,ea_))
        else:
            fout.write("%.6f %.6f %.6f %.6f \n" % (ib*db,ea,ev,fide))
        fout.flush()

        if(ncheck>0): check=(ib%ncheck==0)
        else:         check=False
        psi_QITE,dnorm,xv = QITE_step(H_,psi_QITE,db,xv,check)
        svect_LANZ[ib+1]  = svect_LANZ[ib]+np.log(dnorm)

    fout.write("QITE gs wfn \n")
    print_state(psi_QITE,nbit,fout)
    dump_state(psi_QITE,nbit,'qite.psi')
    dump_lanz_vecs(hvect_LANZ[:nbeta],svect_LANZ[:nbeta],'qlanz.vecs')

    fout.close()


In [40]:
from hamiltonian      import Hubbard,print_Hamiltonian
# from ite              import ITE
# from qite             import QITE
from binary_functions import Bas2Int

# 2-site Hubbard
norb =  2
R    =  1.50
db   =  0.05
bmax =   8.0
U    =   1.0

print("Topology: ")
H = Hubbard(norb,R,U)

print("Hubbard Hamiltonian: ")
print_Hamiltonian(H)

# AFM initial guess
print("AFM initial guess... ")
nspin = 2*norb
psi_0 = np.zeros(2**nspin,dtype=complex)
xvec  = [0]*nspin
for ii in range(0,nspin,4):
    xvec[ii]   = 1
    xvec[ii+3] = 1
xind        = Bas2Int(xvec,2)
psi_0[xind] = 1.0
print("psi_0: ", psi_0)

# print("ITE: ")
# ITE(H,db,bmax,psi0=psi_0)
print("QITE: ")
QITE(H,db,bmax,lanczos=False,psi0=psi_0,ncheck=-1) #10)

Topology: 
>>>>>>> sites  0 1
neighborhood
[0 1]
range(0, 4)
-----
interaction on site  0
indices 0 1 4 1.0
[3, 0, 0, 0]
[0, 3, 0, 0]
[3, 3, 0, 0]
interaction on site  1
indices 2 3 4 1.0
[0, 0, 3, 0]
[0, 0, 0, 3]
[0, 0, 3, 3]
kinetic
spin  0
[1, 3, 1, 0]
[2, 3, 2, 0]
spin  1
[0, 1, 3, 1]
[0, 2, 3, 2]
remember to add the constant correction  -0.5
Hubbard Hamiltonian: 
term  0
active qubits  range(0, 4)
operators: 
IIZZ 0.25
IXZX -0.5
IYZY -0.5
XZXI -0.5
YZYI -0.5
ZZII 0.25
AFM initial guess... 
psi_0:  [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
QITE: 
0 (0.5+0j)
1 0j
2 0j
3 (0.5+0j)
4 0j
5 (-0.5+0j)
6 (-0.5+0j)
7 0j
8 0j
9 (-0.5+0j)
10 (-0.5+0j)
11 0j
12 (0.5+0j)
13 0j
14 0j
15 (0.5+0j)
epsm0:  -2.0615528128088276


In [31]:
import numpy as np
from pyquil.gates import *
from pyquil.noise import estimate_bitstring_probs
from pyquil import Program,get_qc

n = 25
shots = 5000
E = np.zeros([n+1],dtype=complex)
alist = []
def ansatz(p):
    # Construct the ansatz for the initial state
    p += H(0)
#     p += H

def measure(p,ro,idx):
    # We define measure as follow: We pass it an index idx that indicates which pauli expectation we want
    # idx = 0, measure "I"
    # idx = 1, measure "X"
    # idx = 2, measure "Y"
    # idx = 3, measure "Z"
    # I filled in the portion of the code for idx = 0, 1, your task is to fill in the portion for idx = 2 and 3.
    
    if idx == 0:
        print(p)
        return 1
    elif idx == 1:
        qc = get_qc('1q-qvm')
        p += H(0)
        p += MEASURE(0,ro[0])
        print(p)
        exe = qc.compile(p)
        res = qc.run(exe)
        probs = estimate_bitstring_probs(res)
        return probs[0] - probs[1]

    # 1(b) Fill in here
    elif idx == 2:
        qc = get_qc('1q-qvm')
        p += RX(np.pi/2,0)
        p += MEASURE(0,ro[0])
        print(p)
        exe = qc.compile(p)
        res = qc.run(exe)
        probs = estimate_bitstring_probs(res)
        return probs[0] - probs[1]
    elif idx == 3:
        qc = get_qc('1q-qvm')
        p += MEASURE(0,ro[0])
        print(p)
        exe = qc.compile(p)
        res = qc.run(exe)
        probs = estimate_bitstring_probs(res)
        return probs[0] - probs[1]

def measure_energy(nn,alist,shots):
    p = Program()
    p.wrap_in_numshots_loop(shots)
    # ro: readout results: declared memory space with 1 bet of memory to read measurement results
    ro = p.declare('ro','BIT',1)
    # define ansatz
    ansatz(p)
    Energy = measure(p,ro,2)
    return Energy

E[0] = measure_energy(0,alist,shots)
print('Initial energy is: ',E[0])

DECLARE ro BIT[1]
H 0
RX(pi/2) 0
MEASURE 0 ro[0]

Initial energy is:  (0.03160000000000002+0j)
